In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Get Data

In [2]:
dtrain = pd.read_csv('../data/1e7data_train.csv', index_col=0)
dtest = pd.read_csv('../data/1e7data_holdout.csv', index_col=0)
dtrain.head()

,Stable,Norbits_instability,Norbits,Mplanet,RH/a,a1,P1,e1,pomega1,inc1,...,norm_min_e2,norm_min_window10_e2,norm_min_e3,norm_min_window10_e3,norm_min_inc1,norm_min_window10_inc1,norm_min_inc2,norm_min_window10_inc2,norm_min_inc3,norm_min_window10_inc3
Sim_ID,,,,,,,,,,,,,,,,,,,,,
2414,True,1.000000e+07,10000000.0,0.000015,0.021544,0.056553,0.013449,0.008127,-4.462116,0.003942,...,0.000304,0.013332,0.000026,0.035966,0.000702,0.000327,0.000850,0.001853,0.000268,0.000527
4056,False,2.980163e+04,10000000.0,0.000015,0.021544,0.050793,0.011447,0.005201,4.606446,0.009269,...,0.000536,0.085972,0.000398,0.009833,0.000102,0.000159,0.000238,0.007656,0.000095,0.000473
4855,True,1.000000e+07,10000000.0,0.000015,0.021544,0.053113,0.012241,0.004883,0.605644,0.000564,...,0.000176,0.048310,0.000261,0.006232,0.000552,0.025484,0.000111,0.000176,0.000191,0.000011
517,True,1.000000e+07,10000000.0,0.000015,0.021544,0.056243,0.013338,0.008056,1.245784,0.007544,...,0.000050,0.004476,0.000432,0.000578,0.000763,0.000076,0.000403,0.002325,0.000358,0.000026
1520,False,2.241913e+04,10000000.0,0.000015,0.021544,0.048262,0.010603,0.012062,-0.577740,0.005068,...,0.000107,0.004912,0.000071,0.025597,0.000004,0.008796,0.000984,0.001798,0.000431,0.027389


In [36]:
i=4056
df = pd.read_csv("../data/featuresfromshortruns/output%d.csv"%i, names=columns)
df['Lyapunov Time'][0] = 0
df.head()

,Time,Lyapunov Time,a1,P1,e1,pomega1,inc1,Omega1,f1,a2,...,inc2,Omega2,f2,a3,P3,e3,pomega3,inc3,Omega3,f3
0,0.000000,0.000000,0.050788,0.011446,0.005238,4.596614,0.009269,2.392094,-3.965520,0.057271,...,0.000350,1.279788,1.312537,0.063555,0.016022,0.007325,-3.925685,0.009493,-1.889772,-0.754900
1,0.057257,-1.206137,0.050783,0.011444,0.005739,4.292296,0.009282,2.384525,-3.651836,0.057279,...,0.000310,1.281682,2.513768,0.063554,0.016022,0.007500,-3.936032,0.009518,-1.891515,2.879105
2,0.114514,1.350826,0.050765,0.011438,0.006962,3.989307,0.009298,2.379684,-3.327749,0.057264,...,0.000380,0.708299,-2.741458,0.063601,0.016039,0.005090,-3.242974,0.009572,-1.912493,-0.502585
3,0.171772,0.831175,0.050732,0.011427,0.008665,3.908612,0.009306,2.373476,-3.210690,0.057309,...,0.000357,0.670544,-1.606675,0.063597,0.016038,0.005244,-3.262142,0.009596,-1.914293,3.104247
4,0.229029,0.871701,0.050719,0.011422,0.010572,3.894819,0.009308,2.369147,-3.138181,0.057269,...,0.000392,0.362015,-0.380497,0.063674,0.016067,0.008257,-3.043476,0.009578,-1.926638,0.158190


In [3]:
columns = ['Time', 'Lyapunov Time']
for i in ['1', '2', '3']:
    columns += ['a'+i, 'P'+i, 'e'+i, 'pomega'+i, 'inc'+i, 'Omega'+i, 'f'+i]

In [42]:
def fft_samples(df):
    d = {}
    hanning = np.hanning(len(df))
    for c in df.columns[1:]: #ignore time
        d[c] = np.abs(np.fft.rfft(df[c]*hanning))[5:] #skip first couple frequencies, very noisy
    return pd.DataFrame(d).transpose().values #need transpose for X_train.shape = (n_samples,n_colours,len(timeseries))
        
#Each 1D image has length 2000 with 22 channels (or 22 colours)
from keras.utils import np_utils
def get_samples(d, indices, columns):
    X = []
    y = []
    for i in indices:
        sample = pd.read_csv("../data/featuresfromshortruns/output%d.csv"%i, names=columns)
        if len(sample) == 2000:                    #remove runs that are unstable on <10^4 orbits
            #del sample['Time']                     #time is an irrelevant
            #sample['Lyapunov Time'][0] = 0         #first element = inf. Set to 0 to avoid confusion
            #X.append(sample.transpose().values)    #I think I want transpose, X_train.shape = (n_samples,n_colours,len(timeseries))
            X.append(fft_samples(sample))
            y.append(int(d['Stable'].iloc[d.index == i]))
    X = np.asarray(X)
    y = np_utils.to_categorical(np.asarray(y))
    #num_classes = y.shape[1]
    return X, y

In [43]:
#Gathering all samples, this will take a minute
X_train, y_train = get_samples(dtrain, dtrain.index, columns)
X_test, y_test = get_samples(dtest, dtest.index, columns)
num_classes = y_train.shape[1]

## Build Model

In [45]:
from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Dropout, Convolution1D, MaxPooling1D, Flatten, Convolution2D
from keras.constraints import maxnorm
from keras.optimizers import SGD

In [48]:
#create model - Convnet1D
n_filters = 32
filter_length = 100

model = Sequential()
model.add(Convolution1D(n_filters, filter_length, input_shape=(X_test.shape[1], X_test.shape[2]), border_mode='same', activation='relu')) #input_shape=(n_colours, len(timeseries))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_length=2))   #half the image in each direction
model.add(Flatten())
model.add(Dense(50, activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [50]:
# Compile model
epochs = 3
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution1d_1 (Convolution1D)  (None, 22, 32)        3187232     convolution1d_input_1[0][0]      
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 22, 32)        0           convolution1d_1[0][0]            
____________________________________________________________________________________________________
maxpooling1d_1 (MaxPooling1D)    (None, 11, 32)        0           dropout_1[0][0]                  
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 352)           0           maxpooling1d_1[0][0]             
___________________________________________________________________________________________

In [51]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=epochs, batch_size=32)

Train on 3284 samples, validate on 1417 samples
Epoch 1/3
3284/3284 [==============================] - 93s - loss: 0.6339 - acc: 0.6848 - val_loss: 0.6217 - val_acc: 0.6867

## Extra crap to be deleted later

In [22]:
#create model - Convnet2D
model = Sequential()
model.add(Convolution2D(32, 100, 3, input_shape=(1, 2000, 22), border_mode='same', activation='relu'))
model.add(Dropout(0.2))
#model.add(MaxPooling1D(pool_length=2))   #half the image in each direction
#model.add(Convolution1D(32, 100,  activation='relu', border_mode='same', W_constraint=maxnorm(3)))
#model.add(MaxPooling1D(pool_length=2))   #half the image in each direction
model.add(Flatten())
model.add(Dense(50, activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
print X_train.shape
print y_test.shape

In [ ]:
#proper input to keras I think
y_train2 = np_utils.to_categorical(y_train)
print y_train2.shape
print y_train2[0]